In [188]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio
import os 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Dense, Flatten

In [319]:
import glob
c1 = glob.glob('Data/chest_xray/train/*/*')
c2 = glob.glob('Data/chest_xray/test/*/*')

print('# Train images : {} \n# Test images : {}'.format(len(c1), len(c2)))

# Train images : 5216 
# Test images : 624


## Loading, resizing, classifying images

In [298]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
          
def returnLabel(path):
    imgClass = tf.strings.split(path,'/')[-2]
    return 0 if imgClass == 'NORMAL' else 1

def returnImg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    imgClass = returnLabel(path)
    return tf.image.resize(img, (256,256)), imgClass

In [299]:
train_path = 'Data/chest_xray/train'
list_ds = tf.data.Dataset.list_files(str(train_path+'/*/*'))
labeled_ds = list_ds.map(returnImg, num_parallel_calls=AUTOTUNE)
labeled_ds

<ParallelMapDataset shapes: ((256, 256, 3), ()), types: (tf.float32, tf.int32)>

## Model


In [342]:
class AccuracyHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.loss_ = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('accuracy'))
        self.loss_.append(logs.get('loss'))

history = AccuracyHistory()

In [345]:
def nn_model():
    model = Sequential()
    
    model.add(Conv2D(8,5, strides=(5,5), padding='SAME', activation='relu', use_bias=True))
    model.add(AveragePooling2D(pool_size=(2,2), data_format='channels_last'))
    
    model.add(Conv2D(10, 5, activation='relu', use_bias=True))
    model.add(AveragePooling2D(pool_size=(2,2), strides=(1,1), data_format='channels_last'))
    
    model.add(Flatten())
    
    model.add(Dense(120, activation='relu', use_bias=True))
    model.add(Dense(84, activation='relu', use_bias=True))
    model.add(Dense(1, activation='sigmoid', use_bias=True))
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
    
    return model
    

In [360]:
epoch = 25
batch_size = 128
seed = 1 
labeled_data = labeled_ds.batch(5300)
model = nn_model()

model.fit(labeled_data, batch_size=128, epochs=epoch, verbose=1, callbacks=[history])

Epoch 1/25
1/1 [==============================] - 0s 1ms/step - loss: 0.7085 - accuracy: 0.2575
Epoch 2/25
1/1 [==============================] - 0s 2ms/step - loss: 0.5689 - accuracy: 0.7429
Epoch 3/25
1/1 [==============================] - 0s 1ms/step - loss: 0.6200 - accuracy: 0.7429
Epoch 4/25
1/1 [==============================] - 0s 1ms/step - loss: 0.5656 - accuracy: 0.7429
Epoch 5/25
1/1 [==============================] - 0s 1ms/step - loss: 0.5609 - accuracy: 0.7429
Epoch 6/25
1/1 [==============================] - 0s 1ms/step - loss: 0.5684 - accuracy: 0.7429
Epoch 7/25
1/1 [==============================] - 0s 1ms/step - loss: 0.5617 - accuracy: 0.7429
Epoch 8/25
1/1 [==============================] - 0s 1ms/step - loss: 0.5479 - accuracy: 0.7429
Epoch 9/25
1/1 [==============================] - 0s 908us/step - loss: 0.5364 - accuracy: 0.7429
Epoch 10/25
1/1 [==============================] - 0s 1ms/step - loss: 0.5283 - accuracy: 0.7429
Epoch 11/25
1/1 [====================

## Testing

In [361]:
def testing(model, test_dataset):    
    return model.evaluate(test_dataset, verbose = 1, batch_size=128, return_dict=True)
    

In [362]:
test_path = 'Data/chest_xray/test'
test_ds = tf.data.Dataset.list_files(str(train_path+'/*/*'))
test_labeled = test_ds.map(returnImg, num_parallel_calls=AUTOTUNE)
test_labeled = test_labeled.batch(1000)

In [363]:
score = testing(model, test_labeled)

6/6 [==============================] - 18s 3s/step - loss: 0.2416 - accuracy: 0.8982
